In [113]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString 
import requests
import time

# load in data into geoDF
df = pd.read_csv("perth.csv")
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lng, df.lat), crs=4326)

# determine consecutive rows where drove2 flag is the same and group into chunks
df['chunk_drive'] = df['drove2'].ne(df['drove2'].shift()).cumsum()
df_drive = df.groupby('chunk_drive')
# create list of frame chunks where drove2 flag is true
dfs_drive = []
for name, data in df_drive:
    if data["drove2"].iloc[0]:
        dfs_drive.append(data)

# determine consecutive rows where railed2 flag is the same and group into chunks
df['chunk_rail'] = df['railed2'].ne(df['railed2'].shift()).cumsum()
df_rail = df.groupby('chunk_rail')
# create list of frame chunks where railed2 flag is true
dfs_rail = []
for name, data in df_rail:
    if data["railed2"].iloc[0]:
        dfs_rail.append(data)

In [114]:
for chunk in dfs_rail:

    # stack coordinates into API injestible string
    coords = ";".join(chunk.lng.astype(str) + "," + chunk.lat.astype(str))

    # make journey request to osrm
    r = requests.get("http://localhost:5000/route/v1/driving/" + coords, params={
        "geometries": "geojson",
        "steps": "true",
        #"annotations": "distance,duration"
    })

    # decode response and parse into each leg of journey
    legs = r.json()["routes"][0]["legs"]
    parsed_legs = []
    for leg in legs:
        leg_coords = []
        for step in leg["steps"]:
            leg_coords.extend(step["geometry"]["coordinates"])
        parsed_legs.append({"geometry": LineString(leg_coords)}) #, "duration": leg["duration"], "distance": leg["distance"]})
    # add each parsed leg to relevant row in df
    for idx,val in enumerate(parsed_legs):
        df.geometry.iloc[chunk.index[idx]] = val["geometry"]

    # put in a sleep to ease up API requests
    #time.sleep(1)

C:\Users\MikeRulez\AppData\Local\Temp\ipykernel_15404\1280020913.py:23: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df.geometry.iloc[chunk.index[idx]] = val["geometry"]
C:\Users\MikeRulez\AppData\Local\Temp\ipykernel_15404\1280020913.py:23

In [115]:
for chunk in dfs_drive:

    wrapped_flag = False
    # quick check to fix world-wrapping coordinates for Canada
    if chunk.lng.iloc[0] > 180:
        chunk.lng = chunk.lng - 360
        wrapped_flag = True

    # stack coordinates into API injestible string
    coords = ";".join(chunk.lng.astype(str) + "," + chunk.lat.astype(str))

    # make journey request to osrm
    r = requests.get("http://router.project-osrm.org/route/v1/driving/" + coords, params={
        "geometries": "geojson",
        "steps": "true",
        #"annotations": "distance,duration"
    })

    # decode response and parse into each leg of journey
    legs = r.json()["routes"][0]["legs"]
    parsed_legs = []
    for leg in legs:
        leg_coords = []
        for step in leg["steps"]:
            # if we subtracted 180 from long earlier, re add it now
            if wrapped_flag:
                step["geometry"]["coordinates"] = [[x[0]+360,x[1]] for x in step["geometry"]["coordinates"]]

            leg_coords.extend(step["geometry"]["coordinates"])
        parsed_legs.append({"geometry": LineString(leg_coords)}) #, "duration": leg["duration"], "distance": leg["distance"]})

    # add each parsed leg to relevant row in df
    for idx,val in enumerate(parsed_legs):
        df.geometry.iloc[chunk.index[idx]] = val["geometry"]

    # put in a sleep to ease up API requests
    time.sleep(5)

#df

C:\Users\MikeRulez\AppData\Local\Temp\ipykernel_15404\1458635170.py:34: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df.geometry.iloc[chunk.index[idx]] = val["geometry"]
C:\Users\MikeRulez\AppData\Local\Temp\ipykernel_15404\1458635170.py:34

In [116]:
# set last drove2 flag to false due to lack of onward geometry line
for chunk in dfs_drive:
    df.drove2.iloc[chunk.index[-1]] = False

# set last railed2 flag to false due to lack of onward geometry line
for chunk in dfs_rail:
    df.railed2.iloc[chunk.index[-1]] = False

C:\Users\MikeRulez\AppData\Local\Temp\ipykernel_15404\3315004940.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df.drove2.iloc[chunk.index[-1]] = False
C:\Users\MikeRulez\AppData\Local\Temp\ipykernel_15404\3315004940.py:3: SettingWithCo

In [117]:
df.to_file("trip.geojson", driver="GeoJSON")